# Linear regression

files needed = ('sleep75.dta', 'wage1.dta')

This notebook introduces us to the statsmodels package [(docs)](https://devdocs.io/statsmodels/), which provides functions for formulating and estimating statistical models. Econometrics is a prerequisite for this course, so this notebook will not address the models, per se, but will focus on how to take what you learned in econometrics class and use it in python. 

Most of you used STATA in your econometrics course. STATA is a great package for econometrics. Python can do most of what STATA can do, but STATA will have more specialized routines available. As python's popularity is grows the kinds of models you can estimated in grows, too.    

If STATA is your thing, this [page](http://rlhick.people.wm.edu/posts/comparing-stata-and-ipython-commands-for-ols-models.html) on Rob Hicks' website is a nice STATA to python concordance.  

In [ ]:
import pandas as pd                    # for data handling
import numpy as np                     # for numerical methods and data structures
import matplotlib.pyplot as plt        # for plotting
import seaborn as sea                  # advanced plotting

import patsy                           # provides a syntax for specifying models  
import statsmodels.api as sm           # provides statistical models like ols, gmm, anova, etc...
import statsmodels.formula.api as smf  # provides a way to directly spec models from formulas

### Reading Stata data files

Most of you used Wooldridge's textbook in econometrics. I figure you would like to relive those happy times, so we can work through some of the problems in the Wooldridge textbook. 

On the plus side, the data that correspond to the Wooldridge problems are available to download and they are **ALREADY CLEANED.** \[I contemplated adding some junk to the files to make it more interesting...\]

On the minus side, the files are in STATA's .dta format. 

Lucky for use, pandas has a method that [reads stata files](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_stata.html). It also has methods for SQL, SAS,json,...

In [ ]:
# Use pandas read_stata method to get the stata formatted data file into a DataFrame.
sleep = pd.read_stata('sleep75.dta')

# Take a look...so clean!
sleep.head()

In [ ]:
sleep.info()

### Describing models with patsy
The patsy package provides us with a formulaic syntax for defining models that uses strings. The basic syntax is 
```
y ~ x1 + x2
```
which describes the model 

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \epsilon$.

Notice that I did not specify the constant. Patsy takes care of that automatically. Let's start slow and build up the regression, then we will see how to do it all in one shot. 

We start by passing our model, specified in a patsy string to `patsy.dmatrices( )` to create the *design matrices*. Our model is 

$$ sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 age + \epsilon $$.

\[This is in problem 3, chapter 3 or Wooldrigde.\]

In [ ]:
# Pass the model formula and the associated data to create design matrices
y, X = patsy.dmatrices('sleep ~ totwrk + educ + age', sleep)

In [ ]:
# What do we have?
print('X and y are of type:' , type(X), type(y))

In [ ]:
# Peak at X and y
X

In [ ]:
y

So `X` holds the independent variables and `y` holds the dependent variable. Note the addition of the intercept (the column of ones) to the `X` matrix. 

### Building and estimating the model in statsmodels

With the design matrices in hand, we can build **ordinary least squares** model in statsmodels. 

In [ ]:
# Pass design matrices to OLS to spec an ordinary least squares model
sleep_model = sm.OLS(y, X)
type(model)

We can now estimate (fit) the model using the `.fit( )` method of the statsmodel object.

In [ ]:
res = model.fit()  # Estimate the model and store the results in res
type(res)          # What do we have here?


In [ ]:
# To see the summary report
print(res.summary())

The more you work, the less you sleep. I feel better already.

We can retrieve individual results from the RegressionResultsWrapper object. Try res.`TAB` to see what's in there. 

In [ ]:
print('The parameters are:', res.params, '\n')
print('The confidence intervals are:', res.conf_int(), '\n')
print('The r-sqared is:', res.rsquared)

### Directly specifying and estimating models with the formula.api

We have built our model from the ground up
1. Create the design matrices with patsy
2. Create the model with statsmodel
3. Fit the model and obtain results

That was helpful to get a sense of what is going on, but we can do all those steps in one line of code. We just pass the patsy string and the data directly to statsmodels and call fit.

To do this, we use the `statsmodels.formula.api` methods, which we imported as `smf`. The formula api interprets the patsy formula for us, and creates the design matrices. 

In [ ]:
res = smf.ols('sleep ~ totwrk + educ + age', data=sleep).fit()
print(res.summary())

### Data transformations
Patsy can handle common (and less common) regression tasks. Here are a few

#### Interacting variables
Use '\*' to interact two variables. Patsy will also include the two variables individually, too. Below, we interact education an age

$$ sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 age + \beta_4 age\times educ + \epsilon $$.

In [ ]:
res = smf.ols('sleep ~ totwrk + educ*age', data=sleep).fit()
print(res.summary())

#### logs 

We use the `np.log( )` method directly in the patsy syntax. Note that we loaded the numpy package above as np. Below, we use the logarithm of age in the model

$$ sleep = \beta_0 + \beta_1 totwrk + \beta_2 educ + \beta_3 \log(age)  + \epsilon $$.

In [ ]:
res = smf.ols('sleep ~ totwrk + educ + np.log(age)', data=sleep).fit()
print(res.summary())

#### Fixed effects
When I was a kid, we called these dummy variables. Gender is coded {0,1} in the variable male. 

In [ ]:
sleep['male'].head()

In [ ]:
res = smf.ols('sleep ~ totwrk + educ + age + C(male)', data=sleep).fit()
print(res.summary())

The 'T.1.0' notation is a bit confusing in this context. It means it is giving the value for the '1.0' variable. Since we have included a constant, one of the categories gets dropped. 

To see things more clearly, let's recode the male variable. 

In [ ]:
sleep['gender'] = sleep['male'].replace({1.0:'male', 0.0:'female'})

res = smf.ols('sleep ~ totwrk + educ + age + C(gender)', data=sleep).fit()
print(res.summary())

#### No intercept 
Use `-1` to kill the automatic intercept. Try is with our gender data to explicitly recover the female fixed effect. Now you have to do the subtraction yourself to see that males sleep, on average, 87.9933 more hours.

In [ ]:
res = smf.ols('sleep ~ totwrk + educ + age + C(gender) -1', data=sleep).fit()
print(res.summary())

## Practice

Take a few minutes and try the following. Feel free to chat with those around you if you get stuck. The TA and I are here, too. 

Wooldridge problem C2 in chapter 6. 

1. Load wage1.dta
2. Scatter plot log(wage) against educ

3. Estimate 
$$ \log(wage) = \beta_0 + \beta_1 educ + \epsilon$$

4. Scatter plot the residuals against education. The residuals are in the results object from the fit method. 

5. Looks heteroskedastic. We can change the covariance matrix type (which will correct the standard error calculation) using the `cov_type` parameter [(docs)](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.fit.html#statsmodels.regression.linear_model.OLS.fit). The types of covariance matrices are described in the [docs](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html).

Try 'HC3' for your covariance matrix type.

6. Scatter plot the data and add the regression line.


To plot the regression line you will need to create some x data and then apply the parameters. I used something like this
```python
y = [p.Intercept + p.educ*i for i in x]
```
where `p` hold the parameters from my results and x holds a few x data points. 

7. Go back and add the text 'log(w) = a + b*educ' to the northwest corner of your plot. Replace a and b with the estimated parameter values. 

8. Let's add some more regressors. Estimate
$$ \log(wage) = \beta_0 + \beta_1 educ + \beta_2 exper + \beta_3 exper^2 + \beta_4I_m + \epsilon$$

where $I_m$ is a variable equal to 1 if the worker is a married.